In [ ]:
import pandas as pd
import numpy as np
import random as rn
import tensorflow as tf
from tensorflow import keras
import joblib

# keras random seed https://junstar92.tistory.com/98

RANDOM_SEED = 41
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

from keras import optimizers, initializers, losses

EPOCHS = 10000
LEARNING_RATE = 0.00005
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)
INITIALIZER = initializers.HeNormal(seed=RANDOM_SEED)
LOSS = losses.MeanSquaredError()

In [ ]:
# data Load

import joblib

LINEAR = 'linear_'
POLYNOMIAL = 'polynomial_'

encoder_x = joblib.load('C:/Users/js/Desktop/AISPRAK/사회문제 해결형/'+LINEAR+'encoder_x.pkl') 

y = joblib.load('C:/Users/js/Desktop/AISPRAK/사회문제 해결형/'+LINEAR+'y.pkl') 

decoder_x = joblib.load('C:/Users/js/Desktop/AISPRAK/사회문제 해결형/'+LINEAR+'decoder_x.pkl') 



In [ ]:
encoder_x.shape

In [ ]:
decoder_x = decoder_x.astype('float')
decoder_x.shape

In [ ]:
y = y.reshape(34944,72,1)
y.shape

In [ ]:
# Early Stopping

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',         
                   min_delta=0.01,                
                   patience=200,                 
                   verbose=1,
                   restore_best_weights=True)  


# Model

In [ ]:
# https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=chunjein&logNo=221589624838
# https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

In [ ]:
tf.random.set_seed(RANDOM_SEED)
tf.keras.backend.clear_session()

from keras.models import Model
from keras.layers import Input, LSTM, Dense, TimeDistributed

BATCH_SIZE = None

ENCODER_INPUT_DIM = encoder_x.shape[2]
DECODER_INPUT_DIM = decoder_x.shape[2]

ENCODER_TIMESTEP = 48
DECODER_TIMESTEP = 72
LATENT_DIM = 256

# Encoder

encoder_inputs = Input(shape=(ENCODER_TIMESTEP, ENCODER_INPUT_DIM))

lstm_enc = LSTM(LATENT_DIM, activation='relu', return_sequences=True)(encoder_inputs)

_, latent_c, latent_h = LSTM(LATENT_DIM, activation='relu', return_sequences=False, return_state=True)(lstm_enc) # return_sequences = True는 timestep별 출력 , return_state = True, last_hidden_state 출력

encoder_output = [latent_c, latent_h]

# Decoder

decoder_inputs = Input(shape=(DECODER_TIMESTEP, DECODER_INPUT_DIM))

lstm_dec = LSTM(LATENT_DIM, activation='relu', return_sequences=True)(decoder_inputs)

step, latent_c, latent_h = LSTM(LATENT_DIM, activation='relu', return_sequences=True, return_state=True)(lstm_dec,
                                     initial_state=encoder_output)


# Regressor

regressor_outputs = Dense(1024, activation='relu')(step)

regressor_outputs = Dense(1024, activation='relu')(regressor_outputs)


regressor_result = TimeDistributed(layer = Dense(1, activation='linear'))(regressor_outputs)

# regressor_result = TimeDistributed(layer = Dense(1, activation='linear'))(step)
   
model = Model([encoder_inputs, decoder_inputs], regressor_result)

model.summary()

In [ ]:
model.compile(loss=LOSS, optimizer=OPTIMIZER)
model.fit(x = [encoder_x,decoder_x], y = y, epochs=EPOCHS, batch_size=16, callbacks=[es], validation_split=0.1, shuffle=True)

# 추론

In [ ]:
LINEAR = 'linear_'
POLYNOMIAL = 'polynomial_'

PATH = '/content/drive/MyDrive/aispark/social/'

test_encoder_x = joblib.load(PATH+LINEAR+'test_encoder_x.pkl') 

test_decoder_x = joblib.load(PATH+LINEAR+'test_decoder_x.pkl') 

In [ ]:
test_encoder_x.shape, test_decoder_x.shape

In [ ]:
y_pred = model.predict([test_encoder_x,test_decoder_x])
y_pred.shape

In [ ]:
result = pd.read_csv('/content/drive/MyDrive/aispark/social/answer_sample.csv')
result.shape

In [1]:
result['PM2.5'] = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1],1)

NameError: name 'y_pred' is not defined

In [ ]:
result.to_csv('/content/drive/MyDrive/aispark/social/result.csv')